In [2]:
import sys
import pandas as pd
import vectorbt as vbt
import numpy as np
from utils.data_utils import fetch_historical_data
from strategies.bollong import bollong_signals
from config import SYMBOL, INITIAL_CAPITAL, FEES, logger

def run_verification_backtest():
    # Haal data op
    df = fetch_historical_data(SYMBOL)
    if df is None:
        logger.error("Geen data beschikbaar om te verifiëren")
        return

    # Gebruik de specifieke parameters van de optimalisatie
    window = 60
    std_dev = 2.5
    sl_atr_mult = 1.5
    tp_atr_mult = 4.5

    logger.info(f"Verificatie backtest gestart met parameters:")
    logger.info(f"  Window: {window}")
    logger.info(f"  Std Dev: {std_dev}")
    logger.info(f"  SL ATR Mult: {sl_atr_mult}")
    logger.info(f"  TP ATR Mult: {tp_atr_mult}")

    # Genereer signalen
    entries, sl_stop, tp_stop = bollong_signals(df, window, std_dev, sl_atr_mult, tp_atr_mult)

    # Voer backtest uit
    pf = vbt.Portfolio.from_signals(
        close=df['close'],
        entries=entries,
        sl_stop=sl_stop,
        tp_stop=tp_stop,
        init_cash=INITIAL_CAPITAL,
        fees=FEES,
        freq='1D'
    )

    # Toon gedetailleerde resultaten
    logger.info("\n===== BACKTEST RESULTATEN =====")
    logger.info(f"Totaal rendement: {pf.total_return():.2%}")
    logger.info(f"Sharpe ratio: {pf.sharpe_ratio():.2f}")
    logger.info(f"Max drawdown: {pf.max_drawdown():.2%}")
    logger.info(f"Aantal trades: {len(pf.trades)}")

    # Veilig ophalen van trades informatie
    logger.info("\n--- Trade Details ---")
    try:
        trades_df = pf.trades.records_readable
        # Controleer of de verwachte kolomnamen aanwezig zijn
        expected_columns = {'Entry Date', 'Exit Date', 'PnL', 'Return'}
        available_columns = set(trades_df.columns)
        if not expected_columns.issubset(available_columns):
            logger.debug(f"Ongeldige kolomnamen in trades_df: Beschikbaar: {available_columns}, "
                         f"Verwacht: {expected_columns}")
        if len(trades_df) > 0:
            for i, trade in trades_df.iterrows():
                entry_date = trade.get('Entry Date', 'Onbekend')
                exit_date = trade.get('Exit Date', 'Onbekend')
                pnl = trade.get('PnL', 'Onbekend')
                ret = trade.get('Return', 'Onbekend')
                logger.info(f"Trade {i+1}: Entry {entry_date if pd.notna(entry_date) else 'Onbekend'} - "
                            f"Exit {exit_date if pd.notna(exit_date) else 'Open'} | "
                            f"PnL: {pnl:.2f} ({ret:.2%})" if isinstance(ret, float) else f"{ret}")
        else:
            logger.info("Geen trades uitgevoerd")
    except Exception as e:
        logger.error(f"Fout bij het verwerken van trades: {str(e)}")

        # Alternatieve manier om trades te tonen
        logger.info("Probeer alternatieve manier om trades te tonen:")
        try:
            entries_indices = np.where(entries)[0]
            if len(entries_indices) > 0:
                for i, entry_idx in enumerate(entries_indices):
                    entry_date = df.index[entry_idx]
                    entry_price = df.iloc[entry_idx]['close']

                    # Zoek bijbehorende exit indien beschikbaar
                    exit_date = "Onbekend"
                    exit_price = 0
                    pnl = "Onbekend"

                    # Als er sl/tp info is, probeer die te gebruiken
                    if sl_stop is not None or tp_stop is not None:
                        exit_info = "Stop loss of take profit geactiveerd"
                    else:
                        exit_info = "Geen exit informatie beschikbaar"

                    logger.info(f"Trade {i+1}: Entry op {entry_date.date()} @ {entry_price:.2f} | Exit: {exit_info}")
            else:
                logger.info("Geen entries gevonden in de signalen.")
        except Exception as sub_e:
            logger.error(f"Ook alternatieve trade weergave mislukt: {str(sub_e)}")

    # Maandelijkse prestaties
    try:
        monthly_returns = pf.returns().resample('M').sum()
        logger.info("\n--- Maandelijkse Prestaties ---")
        for date, ret in monthly_returns.items():
            if ret != 0:  # Alleen maanden met activiteit tonen
                logger.info(f"{date.strftime('%Y-%m')}: {ret:.2%}")
    except Exception as e:
        logger.error(f"Fout bij het berekenen van maandelijkse prestaties: {str(e)}")

    # Voeg een samenvattingsstatistiek toe
    try:
        stats = pf.stats()
        logger.info("\n--- Samenvattende Statistieken ---")
        logger.info(f"Winst Factor: {stats.get('profit_factor', 'N/A')}")
        logger.info(f"Expectancy: {stats.get('expectancy', 'N/A')}")
        logger.info(f"SQN (System Quality Number): {stats.get('sqn', 'N/A')}")
        logger.info(f"Calmar Ratio: {stats.get('calmar_ratio', 'N/A')}")
    except Exception as e:
        logger.error(f"Fout bij het berekenen van samenvattende statistieken: {str(e)}")

    return pf

if __name__ == "__main__":
    run_verification_backtest()

2025-04-07 18:16:13,977 - INFO - Historische data geladen: 764 rijen
2025-04-07 18:16:13,977 - INFO - Verificatie backtest gestart met parameters:
2025-04-07 18:16:13,977 - INFO -   Window: 60
2025-04-07 18:16:13,977 - INFO -   Std Dev: 2.5
2025-04-07 18:16:13,977 - INFO -   SL ATR Mult: 1.5
2025-04-07 18:16:13,977 - INFO -   TP ATR Mult: 4.5
2025-04-07 18:16:13,977 - INFO - Aantal LONG signalen: 20
2025-04-07 18:16:13,993 - INFO - 
===== BACKTEST RESULTATEN =====
2025-04-07 18:16:13,993 - INFO - Totaal rendement: 28.54%
2025-04-07 18:16:13,993 - INFO - Sharpe ratio: 2.15
2025-04-07 18:16:13,993 - INFO - Max drawdown: -5.10%
2025-04-07 18:16:13,993 - INFO - Aantal trades: 4
2025-04-07 18:16:13,993 - INFO - 
--- Trade Details ---
2025-04-07 18:16:14,009 - INFO - Trade 1: Entry Onbekend - Exit Onbekend | PnL: 942.71 (9.43%)
2025-04-07 18:16:14,009 - INFO - Trade 2: Entry Onbekend - Exit Onbekend | PnL: 457.68 (4.18%)
2025-04-07 18:16:14,009 - INFO - Trade 3: Entry Onbekend - Exit Onbeken